Messing with potential uses of Q learning - reached a dead end with table construction and rewarding 

In [1]:
from numpy import *
import cmath
from random import randrange

Want machine to obtain the values of the elements in this array:

In [5]:
CNOT13 = array([[1,0,0,0,0,0,0,0],[0,1,0,0,0,0,0,0],[0,0,1,0,0,0,0,0],[0,0,0,1,0,0,0,0],[0,0,0,0,0,1,0,0],[0,0,0,0,1,0,0,0],[0,0,0,0,0,0,0,1],[0,0,0,0,0,0,1,0]])
CNOT12 = array([[1,0,0,0,0,0,0,0],[0,1,0,0,0,0,0,0],[0,0,1,0,0,0,0,0],[0,0,0,1,0,0,0,0],[0,0,0,0,0,0,1,0],[0,0,0,0,0,0,0,1],[0,0,0,0,1,0,0,0],[0,0,0,0,0,1,0,0]])
(CNOT13.dot(CNOT12))
CNOT12

array([[1, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0]])

In order to turn 

In [6]:
a = 1
b = 1
one = array([[int(0)], [int(1)]])
zero = array([[int(1)], [int(0)]])
psi = a*zero + b*one

psi3 = kron((kron(psi,zero)), zero)
psi3

array([[1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0]])

Into:

In [7]:
b*(kron((kron(one,one)),one)) + a*(kron((kron(zero,zero)),zero))

array([[1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1]])

i.e. input psi * (unknown matrix) = output encoded psi

In [47]:
right = (CNOT13.dot(CNOT12)).dot(psi3)
right

array([[1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1]])

Look at probabilities

- First initialise an Identity matrix

In [11]:
matrix = zeros((8,8))

In [12]:
for i in range(0,8):
    matrix[i,i] = 1

In [13]:
matrix

array([[1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.]])

Define a metric for calculating the similarity between the input array when multiplied by the guessed matrix and the output we expect - here we just compare each element of the 8x1 output

In [16]:
def probability(guess, right):
    summ = 0
    for i in range(0,8):
        diff = (guess[i] - right[i])**2
        summ+= diff
    score = 1 - (summ)/8
    return(score)        

In [17]:
val = float(probability(guess, right))
val

0.75

Trying with applying gates

- Define gates first, we will be using 7 in total for now

In [2]:
Toff = array([[1,0,0,0,0,0,0,0],[0,1,0,0,0,0,0,0],[0,0,1,0,0,0,0,0],[0,0,0,1,0,0,0,0],[0,0,0,0,1,0,0,0],[0,0,0,0,0,1,0,0],[0,0,0,0,0,0,0,1],[0,0,0,0,0,0,1,0]])

In [24]:
X = array([[0 , 1], [1, 0]])
Iden = array([[1 , 0], [0, 1]])
Iden3 = array([[1,0,0,0,0,0,0,0],[0,1,0,0,0,0,0,0],[0,0,1,0,0,0,0,0],[0,0,0,1,0,0,0,0],[0,0,0,0,1,0,0,0],[0,0,0,0,0,1,0,0],[0,0,0,0,0,0,1,0],[0,0,0,0,0,0,0,1]])

In [9]:
X1 = kron(kron(X, Iden), Iden)
X2 = kron(kron(Iden, X), Iden)
X3 = kron(kron(Iden, Iden), X)

Insert them into an iterable list

In [37]:
list2 = [Iden3,X1,X2,X3,Toff,CNOT13,CNOT12]

If we obtain a matrix that gives the output we want, add it to 'sums'. sums then displays the elements of the matrix that are most likely to give this correct return

In [50]:
sums = zeros((8,8))
for i in range(0,7):
    matrixdup = matrix.copy()
    new = array(list2[i]).dot(matrixdup)
    for j in range(0,7):
        new1 = new.copy()
        new2 = array(list2[j]).dot(new)
        for k in range(0,7):
            new3 = new2.copy()
            new4 = array(list2[k]).dot(new3)  
            guess0 = new4.dot(psi3)
            val = float(probability(guess0, right))
            #probs[i,j] = val
            if int(val) == 1:
                sums+= new4

In [51]:
sums

array([[13.,  0.,  0.,  0.,  3.,  0.,  0.,  0.],
       [ 0., 13.,  0.,  0.,  0.,  3.,  0.,  0.],
       [ 0.,  0., 13.,  0.,  0.,  0.,  3.,  0.],
       [ 0.,  0.,  0., 13.,  0.,  0.,  0.,  3.],
       [ 0.,  0.,  1.,  2.,  0.,  0.,  6.,  7.],
       [ 0.,  0.,  2.,  1.,  0.,  0.,  7.,  6.],
       [ 0.,  3.,  0.,  0.,  0., 13.,  0.,  0.],
       [ 3.,  0.,  0.,  0., 13.,  0.,  0.,  0.]])

Q learning

- Create a dataframe of all possible states (matrices) after 3 actions along with their similarity metric 

In [276]:
import pandas as pd
Q_table = pd.DataFrame()

In [277]:
for i in range(0,7):
    matrixdup = matrix.copy()
    new = array(list2[i]).dot(matrixdup)
    for j in range(0,7):
        new1 = new.copy()
        new2 = array(list2[j]).dot(new)
        for k in range(0,7):
            new3 = new2.copy()
            new4 = array(list2[k]).dot(new3)
            guess0 = new4.dot(psi3)
            prob = float(probability(guess0, right))
            reward = 0
            for m in range(0,7):
                gate = m
                Q = pd.DataFrame({'state':[new4], 'ex':i, 'indicator1':j, 'indicator2':k, 'gate':gate, 'Q':0, 'reward':reward, 'prob':prob}, index=[0])
                Q_table = Q_table.append(Q)
Q_table = Q_table.reset_index(drop=True)

In [278]:
Q_table.loc[Q_table['prob']==1]

,state,ex,indicator1,indicator2,gate,Q,reward,prob
287,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0....",0,5,6,0,0,0,1.0
288,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0....",0,5,6,1,0,0,1.0
289,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0....",0,5,6,2,0,0,1.0
290,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0....",0,5,6,3,0,0,1.0
291,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0....",0,5,6,4,0,0,1.0
...,...,...,...,...,...,...,...,...
2305,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0....",6,5,0,2,0,0,1.0
2306,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0....",6,5,0,3,0,0,1.0
2307,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0....",6,5,0,4,0,0,1.0
2308,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0....",6,5,0,5,0,0,1.0


In [280]:
import time
from IPython.display import clear_output

In [284]:
input_table = Q_table.copy()
gamma = 0.8
num_repeats = 2
start_time = time.time()

Begin with every initial state after 2 actions, for each: find the collection of rows from input state that are equal to this matrix. then iterate through these and apply a further matrix (one from the selection of actions) and find this result in the input matrix. If the result has a better similarity metric than the matrix we had before, the reward is 1, if it is worse, reward is -1, 0 if the same. Q values are then calculated based on this difference and previous iterations 

In [293]:
output_metric_table = pd.DataFrame()
for repeats in range(0,num_repeats):
    clear_output(wait=True)
    state_sub_full = pd.DataFrame()
    
    
    for i in range(0,7):
        matrixdup = matrix.copy()
        new = array(list2[i]).dot(matrixdup)
        for k in range(0,7):
            new3 = new.copy()
            new4 = array(list2[k]).dot(new3)
            
            state_sub = pd.DataFrame(columns = ['state', 'ex', 'indicator1', 'indicator2', 'gate', 'Q', 'reward', 'prob'])
            for l in range(0,(len(input_table))):
                if allclose(input_table.iloc[l]['state'], new4) == True:
                    #state_sub = input_table['state'][i]
                    state_sub = state_sub.append(input_table.iloc[[l]])
            print('state_sub')
            
            Q_sub_list = pd.DataFrame()
            print(len(state_sub))
            for n in range(0,(len(state_sub))):
                
                
                for z in range(0,7):
                    if(state_sub.iloc[n]['gate']== z):
                        apply = array(list2[z])
                #print(apply)
                state_sub1 = pd.DataFrame(columns = ['state', 'ex', 'indicator1', 'indicator2', 'gate', 'Q', 'reward', 'prob'])
                for l in range(0,(len(input_table))):
                    if allclose(input_table.iloc[l]['state'], apply.dot(new4)) == True:
                        #state_sub = input_table['state'][i]
                        state_sub1 = state_sub1.append(input_table.iloc[[l]])
                print('state sub1 made')        
                if max(state_sub1['prob']) > max(state_sub['prob']):
                    reward = 1
                elif max(state_sub1['prob']) < max(state_sub['prob']):
                    reward = -1
                else:
                    reward=0
                print('r =' + str(reward))
                Q = (reward + gamma*max(state_sub1['Q']))
                Q_sub_list = Q_sub_list.append(pd.DataFrame({'Q':Q}, index = [n]))
                print('Q = ' + str(Q))
                print('Q calculated for: '+ str(n))
            state_sub['Q'] = Q_sub_list['Q']
            #state_sub_full = state_sub_full.append(state_sub)
            state_sub_full = pd.merge(input_table,state_sub,how='right', on = ['indicator1','indicator2'], suffixes=('_old','_new'))
                                      
            print(str(k) +' initial matrix done')
        print(i)    
    input_table = state_sub_full.copy()
    print("Repeats completed: ", round((repeats+1)/num_repeats,2)*100, "%")
    
end_time = time.time()

print("total time taken this loop: ", round((end_time - start_time)/60,2), " minutes")
    

KeyError: 'state'

In [248]:
for l in range(0,(len(Q_table_VI_3))):
    max(Q_table_VI_3[(Q_table_VI_3.iloc[l]['indicator2'] == 1) & (Q_table_VI_3.iloc[l]['indicator3'] == 1)]['Q'])

KeyError: False

In [250]:
max(Q_table_VI_3[(Q_table_VI_3['indicator2'] == 1) & (Q_table_VI_3['indicator3'] == 1)]['Q'])

0.0

Optimal

In [268]:
Q_table_VI_3 = state_sub_full.copy()

optimal_action = pd.DataFrame(columns = ['state', 'indicator2', 'indicator3', 'gate'])

In [271]:
len(Q_table_VI_3)

260

In [270]:
Q_table_VI_3 = Q_table_VI_3.dropna()

In [267]:
optimal_action_list = pd.DataFrame()

for i in range(0,7):
    matrixdup = matrix.copy()
    new = array(list2[i]).dot(matrixdup)
    for k in range(0,7):
        new3 = new.copy()
        new4 = array(list2[k]).dot(new3)
        
        row = Q_table_VI_3[(Q_table_VI_3['indicator2'] == int(i)) & (Q_table_VI_3['indicator3'] == int(k)) & (Q_table_VI_3['Q']== max(Q_table_VI_3[(Q_table_VI_3['indicator2'] == int(i)) & (Q_table_VI_3['indicator3'] == int(k))]['Q']))]
        #gate = row['gate']
        
        #optimal_action = pd.DataFrame({'state':[new4], 'indicator2':i, 'indicator3':k, 'gate':gate})
        optimal_action_list = optimal_action_list.append(row)
optimal_action_list

ValueError: max() arg is an empty sequence